This Notebook covers the basics of SQL. If you are opening it in Instabase, you can edit and run queries against the database, which is already running on Instabase servers. In other case, you should also make sure that you are comfortable using SQL through `psql` (see the [Setup Instructions](https://github.com/umddb/cmsc424-fall2016/tree/master/project0) to get started with that).

The server is already running. We connect to it using the special commands provided by Instabase for this purpose.

In [1]:
ib.connect_db('ib://amolvdeshpande/CMSC424/databases/umd-cmsc424')

NameError: name 'ib' is not defined

We can now run SQL commands using `magic` commands, which is an extensibility mechanism provided by Jupyter. %sql is for single-line commands, whereas %%sql allows us to do multi-line SQL commands.

In [71]:
%sql SELECT id FROM instructor;

,id
0,10101
1,12121
2,15151
3,22222
4,32343
5,33456
6,45565
7,58583
8,76543
9,76766


One drawback of this way of accessing the database is that we can only run valid SQL -- the commands like `\d` provided by `psql` are not available to us. Instead, we will need to query the system catalog (metadata) directly. The first command below is equivalent to `\d`, whereas the second one is similar to `\d instructor`.

In [72]:
%%sql
SELECT table_schema, table_name FROM information_schema.tables
    WHERE table_type = 'BASE TABLE' AND
    table_schema NOT IN ('pg_catalog', 'information_schema', 'priv');

,table_schema,table_name
0,public,department
1,public,section
2,public,course
3,public,prereq
4,public,classroom
5,public,takes
6,public,instructor
7,public,teaches
8,public,student
9,public,advisor


In [73]:
%%sql
SELECT column_name, data_type
    FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'instructor';

,column_name,data_type
0,id,character varying
1,name,character varying
2,dept_name,character varying
3,salary,numeric


### University Database
Below we will use the University database from the class textbook. The University Dataset is the same as the one discussed in the book, and contains randomly populated information about the students, courses, and instructors in a university. The schema diagram for the database is as follows:
<center><img src="https://github.com/umddb/cmsc424-fall2015/raw/master/postgresql-setup/university.png" width=800px></center>

### Select Queries
Let's start with the most basic queries. The following query reports the courses with titles containing Biology.

In [74]:
%sql select * from course where title like '%Biology%';

,course_id,title,dept_name,credits
0,BIO-101,Intro. to Biology,Biology,4
1,BIO-399,Computational Biology,Biology,3


There are two  courses. How many students are enrolled in the first one (ever)? What about in Summer 2009?

In [75]:
%sql select * from takes where course_id = 'BIO-101';

,id,course_id,sec_id,semester,year,grade
0,98988,BIO-101,1,Summer,2009,A


In [76]:
%sql select * from takes where course_id = 'BIO-101'  and year = 2009 and semester = 'Summer';

,id,course_id,sec_id,semester,year,grade
0,98988,BIO-101,1,Summer,2009,A


### Aggregates

Count the number of instructors in Finance

In [77]:
%sql select count(*) from instructor where dept_name = 'Finance';

,count
0,2


Find the instructor(s) with the highest salary. Note that using a nested "subquery" (which first finds the maximum value of the salary) as below is the most compact way to write this query.

In [78]:
%%sql 
select *
from instructor
where salary = (select max(salary) from instructor);

,id,name,dept_name,salary
0,22222,Einstein,Physics,95000.00


### Joins and Cartesian Product 
To find building names for all instructors, we must do a join between two relations.

In [79]:
%%sql
select name, instructor.dept_name, building
from instructor, department
where instructor.dept_name = department.dept_name;

,name,dept_name,building
0,Srinivasan,Comp. Sci.,Taylor
1,Wu,Finance,Painter
2,Mozart,Music,Packard
3,Einstein,Physics,Watson
4,El Said,History,Painter
5,Gold,Physics,Watson
6,Katz,Comp. Sci.,Taylor
7,Califieri,History,Painter
8,Singh,Finance,Painter
9,Crick,Biology,Watson


Since the join here is a equality join on the common attributes in the two relations, we can also just do:

In [80]:
%%sql 
select name, instructor.dept_name, building
from instructor natural join department;

,name,dept_name,building
0,Srinivasan,Comp. Sci.,Taylor
1,Wu,Finance,Painter
2,Mozart,Music,Packard
3,Einstein,Physics,Watson
4,El Said,History,Painter
5,Gold,Physics,Watson
6,Katz,Comp. Sci.,Taylor
7,Califieri,History,Painter
8,Singh,Finance,Painter
9,Crick,Biology,Watson


On the other hand, just doing the following (i.e., just the Cartesian Product) will lead to a large number of tuples, most of which are not meaningful.

In [81]:
%%sql
select name, instructor.dept_name, building
from instructor, department;

,name,dept_name,building
0,Srinivasan,Comp. Sci.,Watson
1,Wu,Finance,Watson
2,Mozart,Music,Watson
3,Einstein,Physics,Watson
4,El Said,History,Watson
5,Gold,Physics,Watson
6,Katz,Comp. Sci.,Watson
7,Califieri,History,Watson
8,Singh,Finance,Watson
9,Crick,Biology,Watson


### Renaming using "as"
**as** can be used to rename tables and simplify queries:

In [82]:
%%sql
select distinct T.name
from instructor as T, instructor as S  
where T.salary > S.salary and S.dept_name = 'Biology';

,name
0,Wu
1,Gold
2,Katz
3,Singh
4,Brandt
5,Kim
6,Einstein


**Self-joins** (where two of the relations in the from clause are the same) are impossible without using `as`. The following query associates a course with the pre-requisite of one of its pre-requisites. There is no way to disambiguate the columns without some form of renaming.

In [83]:
%%sql
select p1.course_id, p2.prereq_id as pre_prereq_id
from prereq p1, prereq p2
where p1.prereq_id = p2.course_id;

,course_id,pre_prereq_id


The small University database doesn't have any chains of this kind. You can try adding a new tuple using: 

```insert into prereq values ('CS-101', 'PHY-101');```

This won't work here because the database is mounted in read-only mode, but you should try this your own database.

### Set Operations
*Union* operation can be used to combine information from two tables (from Section 3.5.1).

In [84]:
%%sql
select course_id
from section
where semester = 'Fall' and year= 2009
union 
select course_id
from section
where semester = 'Spring' and year= 2010;

,course_id
0,CS-101
1,CS-315
2,CS-319
3,CS-347
4,FIN-201
5,HIS-351
6,MU-199
7,PHY-101


### Aggregation with Grouping (Section 7.4.2)

In [85]:
%%sql
select dept_name, avg(salary) as avg_salary
from instructor
group by dept_name;

,dept_name,avg_salary
0,Comp. Sci.,77333.333333333333
1,Elec. Eng.,80000.000000000000
2,History,61000.000000000000
3,Music,40000.000000000000
4,Finance,85000.000000000000
5,Physics,91000.000000000000
6,Biology,72000.000000000000


You can use `having` to filter out groups. The following query only returns the average salary for departments with more than 2 instructors.

In [86]:
%%sql
select dept_name, avg(salary) as avg_salary
from instructor
group by dept_name
having count(*) > 2;

,dept_name,avg_salary
0,Comp. Sci.,77333.333333333333


### WITH
In many cases you might find it easier to create temporary tables, especially for queries involving finding "max" or "min". This also allows you to break down the full query and makes it easier to debug. It is preferable to use the WITH construct for this purpose. The syntax and support differs across systems, but here is the link to PostgreSQL: http://www.postgresql.org/docs/9.0/static/queries-with.html

These are also called Common Table Expressions (CTEs).

The following query is from Section 3.8.6.

In [87]:
%%sql
with max_budget(value) as (
select max(budget)
from department
)
select budget
from department, max_budget
where department.budget = max_budget.value;

,budget
0,120000.00


### LIMIT
PostgreSQL allows you to limit the number of results displayed which
is useful for debugging etc. Here is an example.

In [88]:
%sql select * from instructor limit 2;

,id,name,dept_name,salary
0,10101,Srinivasan,Comp. Sci.,65000.00
1,12121,Wu,Finance,90000.00


### Try your own queries
Feel free to use the cells below to write new queries. You can also just modify the above queries directly if you'd like.